# Decoder masked scaled dot-product attention

Vamos a volver a recordar la arquitectura y la fórmula del `Scaled Dot-Product Attention`

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention.png" alt="Scaled_Dot-Product_Attention">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Ya explicamos todo menos la parte de la máscara, por lo que vamos a hacer un pequeño recordatorio

## MatMul

Al igual que en el encoder, en el decoder tanto `Q`, como `K` como `V` son la misma matriz

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_decoder_masked_multi_head_attention.png" alt="Multi-Head Attention" style="width:425px;height:626px;">
</div>

Así que en el primer `MatMul` en realidad se va a hacer una multiplicación de la matriz de entrada que llamaremos `X` consigo misma traspuesta

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_first_MatMul.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Dijimos que la matriz `X` se compone del conjunto de vectores de embeddings de los tokens de la frase

$$X = \begin{pmatrix}
v_1 \\
v_2 \\
\vdots\\
v_m \\
\end{pmatrix}$$

Donde `m` es el número de tokens de la frase

Cada vector va a tener tantos elementos como las dimensiones de nuestro embedding, supongamos que es `n`, por tanto

$$X = \begin{pmatrix}
v_{1,1} & v_{1,2} & \cdots & v_{1,n} \\
v_{2,1} & v_{2,2} & \cdots & v_{2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{m,1} & v_{m,2} & \cdots & v_{m,n} \\
\end{pmatrix}$$

Así que la multiplicación de `X` consigo misma transpuesta es

$$X \cdot X^T = \begin{pmatrix}
v_1 \cdot v_1 & v_1 \cdot v_2 & \cdots & v_1 \cdot v_m \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & v_2 \cdot v_m \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix}$$

<div style="text-align:center;">
  <img src="Imagenes/Transformer - matmul.png" alt="MatMul">
</div>

La multiplicación sera una multiplicación de matrices de dimensiones $\left(m \times n\right) \cdot \left(n \times m\right)$ que dará como resultado una matriz de tamaño $\left(m \times m\right)$ donde `m` era el número de tokens de la frase

## Scale

A continuación se dividía entre la raiz de la dimensión de embeddings porque es un tipo de normalización llamada `norma L2`.

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_scale.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Así que nos queda

$$
\text{Scale} = \frac{1}{\sqrt{d_k}} \cdot \left( X \cdot X^T \right) = \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_1 \cdot v_1 & v_1 \cdot v_2 & \cdots & v_1 \cdot v_m \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & v_2 \cdot v_m \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix}
$$

## Softmax

Si no aplicamos la máscara tendriamos que hacer la `Softmax`

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_softmax.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Por lo que nos quedaría una matriz así

$$
\text{Softmax} = \text{softmax}\left( \frac{1}{\sqrt{d_k}} \cdot \left( X \cdot X^T \right) \right) = \text{softmax}\left( \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_1 \cdot v_1 & v_1 \cdot v_2 & \cdots & v_1 \cdot v_m \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & v_2 \cdot v_m \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix} \right)
$$

La cual podemos simplemente suponer como porcentajes de atención

$$
\text{Softmax} = \begin{pmatrix}
p_{1,1} & p_{1,2} & \cdots & p_{1,m} \\
p_{2,1} & p_{2,2} & \cdots & p_{2,m} \\
\vdots & \vdots & \ddots & \vdots \\
p_{m,1} & p_{m,2} & \cdots & p_{m,m} \\
\end{pmatrix}
$$

## MatMul

Al aplicar el último `MatMul`

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_second_MatMul.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

En realidad lo que tenemos es la siguiente matriz

$$
\text{Matmul} = \begin{pmatrix}
p_{1,1} & p_{1,2} & \cdots & p_{1,m} \\
p_{2,1} & p_{2,2} & \cdots & p_{2,m} \\
\vdots & \vdots & \ddots & \vdots \\
p_{m,1} & p_{m,2} & \cdots & p_{m,m} \\
\end{pmatrix} \cdot \begin{pmatrix}
v_1 \\
v_2 \\
\vdots\\
v_m \\
\end{pmatrix} = \begin{pmatrix}
p_{1,1}v_1 + p_{1,2}v_2 + \cdots + p_{1,m}v_m \\
p_{2,1}v_1 + p_{2,2}v_2 + \cdots + p_{2,m}v_m \\
\vdots \\
p_{m,1}v_1 + p_{m,2}v_2 + \cdots + p_{m,m}v_m \\
\end{pmatrix}
$$

Volvemos a recordar cómo era la matriz de entrada

$$X = \begin{pmatrix}
v_1 \\
v_2 \\
\vdots\\
v_m \\
\end{pmatrix} = \begin{pmatrix}
v_{1,1} & v_{1,2} & \cdots & v_{1,n} \\
v_{2,1} & v_{2,2} & \cdots & v_{2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{m,1} & v_{m,2} & \cdots & v_{m,n} \\
\end{pmatrix}$$

Es decir, la primera fila correspondía al embedding del primer token, la segunda fila al embedding del segundo token y así sucesivamente hasta la última fila que corresponde al embedding del último token.

Por lo que si volvemos a ver el resultado de `Scaled Dot-Product Attention` vemos que la matriz

$$
\text{Matmul} = \begin{pmatrix}
p_{1,1} & p_{1,2} & \cdots & p_{1,m} \\
p_{2,1} & p_{2,2} & \cdots & p_{2,m} \\
\vdots & \vdots & \ddots & \vdots \\
p_{m,1} & p_{m,2} & \cdots & p_{m,m} \\
\end{pmatrix} \cdot \begin{pmatrix}
v_1 \\
v_2 \\
\vdots\\
v_m \\
\end{pmatrix} = \begin{pmatrix}
p_{1,1}v_1 + p_{1,2}v_2 + \cdots + p_{1,m}v_m \\
p_{2,1}v_1 + p_{2,2}v_2 + \cdots + p_{2,m}v_m \\
\vdots \\
p_{m,1}v_1 + p_{m,2}v_2 + \cdots + p_{m,m}v_m \\
\end{pmatrix}
$$

Representa que

 * La primera fila (que representaría el primer token) se corresponde a la suma de probabilidades de atención del primer token con el resto de tokens por los embeddings del resto de tokens
 * La segunda fila (que representaría al segundo token) se corresponde a la suma de probabilidades de atención del segundo token con el resto de tokens por los embeddings del resto de tokens
 * Así sucesivamente, hasta la última fila (que representaría al último token) que se corresponde a la suma de probabilidades del último token con el resto de tokens por los embeddings del resto de tokens

## Enmascaramiento

Realizamos las mismas operaciones, pero ahora aplicando la máscara. Veamos ahora qué ocurre cuando se aplica el enmascaramiento

### Primer MatMul

Después del primer `MatMul` seguimos teniendo el mismo resultado

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_first_MatMul.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Tenemos una matriz `X`

$$X = \begin{pmatrix}
v_1 \\
v_2 \\
\vdots\\
v_m \\
\end{pmatrix} = \begin{pmatrix}
v_{1,1} & v_{1,2} & \cdots & v_{1,n} \\
v_{2,1} & v_{2,2} & \cdots & v_{2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{m,1} & v_{m,2} & \cdots & v_{m,n} \\
\end{pmatrix}$$

Que al multiplicarse por ella misma traspuesta queda

$$X \cdot X^T = \begin{pmatrix}
v_1 \cdot v_1 & v_1 \cdot v_2 & \cdots & v_1 \cdot v_m \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & v_2 \cdot v_m \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix}$$

<div style="text-align:center;">
  <img src="Imagenes/Transformer - matmul.png" alt="MatMul">
</div>

### Scale

Se aplica la `norma L2`

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_scale.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Y queda la siguiente matriz

$$
\text{Scale} = \frac{1}{\sqrt{d_k}} \cdot \left( X \cdot X^T \right) = \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_1 \cdot v_1 & v_1 \cdot v_2 & \cdots & v_1 \cdot v_m \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & v_2 \cdot v_m \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix}
$$

### Mask

Ahora sí se aplica el enmascaramiento

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_mask.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Este enmascaramiento consiste en que nos quedemos con la matriz resultante de la diagonal y todos los elemetos por debajo de esa diagonal, es decir, algo así

$$
\text{Masked} = \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_1 \cdot v_1 & 0 & \cdots & 0 \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix}
$$

Lo importante es el resultado que se quiere obtener, y quédate con eso, que es lo importate. Nos queremos quedar con la diagonal y todos los elementos por debajo de ella

Para hacer eso se podría multiplicar elemento a elemento la matriz que sale del `Scale` por la matriz triangular inferior

$$
\begin{pmatrix}
1 & 0 & \cdots & 0 \\
1 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
1 & 1 & \cdots & 1 \\
\end{pmatrix}
$$

Es decir, obtendríamos

$$
\text{Masked} = \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_1 \cdot v_1 & v_1 \cdot v_2 & \cdots & v_1 \cdot v_m \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & v_2 \cdot v_m \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix} x \begin{pmatrix}
1 & 0 & \cdots & 0 \\
1 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
1 & 1 & \cdots & 1 \\
\end{pmatrix} =  \\
= \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_1 \cdot v_1 & 0 & \cdots & 0 \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix}
$$

Pero como esta operación no es deribable (para luego aplicar el descenso del gradiente), lo que se hace es sumarle una matriz donde todos los elementos de la diagonal y por debajo de ella son 0 y los elementos de encima de la diagonal son $-\infty$

$$
\begin{pmatrix}
0 & -\infty & \cdots & -\infty \\
0 & 0 & \cdots & -\infty \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 0 \\
\end{pmatrix}
$$

Así al hacer la suma obtendremos

$$
\text{Masked} = \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_1 \cdot v_1 & v_1 \cdot v_2 & \cdots & v_1 \cdot v_m \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & v_2 \cdot v_m \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix} + \begin{pmatrix}
0 & -\infty & \cdots & -\infty \\
0 & 0 & \cdots & -\infty \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 0 \\
\end{pmatrix} =  \\
= \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_1 \cdot v_1 & -\infty & \cdots & -\infty \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & -\infty \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix}
$$

Y al hacer la siguiente `Softmax` todos los $-\infty$ se convertiran en $0$, ya que $e^{-\infty} = 0$

### Softmax

Ahora tenemos que hacer la `Softmax`

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_softmax.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Por lo que nos quedaría una matriz así

$$
\text{Softmax} = \text{softmax}\left( \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_1 \cdot v_1 & -\infty & \cdots & -\infty \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & -\infty \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix} \right)
$$

La cual podemos simplemente suponer como porcentajes de atención, en la cual los porcentajes de la parte superior de la matriz son $0$

$$
\text{Softmax} = \begin{pmatrix}
p_{1,1} & 0 & \cdots & 0 \\
p_{2,1} & p_{2,2} & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
p_{m,1} & p_{m,2} & \cdots & p_{m,m} \\
\end{pmatrix}
$$

### MatMul

Al aplicar el último `MatMul`

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_second_MatMul.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

En realidad lo que tenemos es la siguiente matriz

$$
\text{Matmul} = \begin{pmatrix}
p_{1,1} & 0 & \cdots & 0 \\
p_{2,1} & p_{2,2} & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
p_{m,1} & p_{m,2} & \cdots & p_{m,m} \\
\end{pmatrix} \cdot \begin{pmatrix}
v_1 \\
v_2 \\
\vdots\\
v_m \\
\end{pmatrix} = \begin{pmatrix}
p_{1,1}v_1 + 0 \cdot v_2 + \cdots + 0 \cdot v_m \\
p_{2,1}v_1 + p_{2,2}v_2 + \cdots + 0 \cdot v_m \\
\vdots \\
p_{m,1}v_1 + p_{m,2}v_2 + \cdots + p_{m,m}v_m \\
\end{pmatrix} = \\
= \begin{pmatrix}
p_{1,1}v_1 \\
p_{2,1}v_1 + p_{2,2}v_2 \\
\vdots \\
p_{m,1}v_1 + p_{m,2}v_2 + \cdots + p_{m,m}v_m \\
\end{pmatrix}
$$

Volvemos a recordar cómo era la matriz de entrada

$$X = \begin{pmatrix}
v_1 \\
v_2 \\
\vdots\\
v_m \\
\end{pmatrix} = \begin{pmatrix}
v_{1,1} & v_{1,2} & \cdots & v_{1,n} \\
v_{2,1} & v_{2,2} & \cdots & v_{2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{m,1} & v_{m,2} & \cdots & v_{m,n} \\
\end{pmatrix}$$

Es decir, la primera fila correspondía al embedding del primer token, la segunda fila al embedding del segundo token y así sucesivamente hasta la última fila que corresponde al embedding del último token.

Por lo que si volvemos a ver el resultado de `Scaled Dot-Product Attention` pero aplicando el enmascaramiento vemos que la matriz es

$$
\text{Matmul} = \begin{pmatrix}
p_{1,1}v_1 \\
p_{2,1}v_1 + p_{2,2}v_2 \\
\vdots \\
p_{m,1}v_1 + p_{m,2}v_2 + \cdots + p_{m,m}v_m \\
\end{pmatrix}
$$

Representa que

 * La primera fila (que representaría el primer token) se corresponde a la probabilidad de atención del primer token consigo mismo por el embedding del primer token
 * La segunda fila (que representaría al segundo token) se corresponde a la suma de la probabilidad de atención del segundo token con la probabilidad de atención del primer token por el embedding del primer token, más la probabilidad de de atención del segundo token consigo mismo por el embedding del segundo token
 * Así sucesivamente, hasta la última fila (que representaría al último token) que se corresponde a la suma de probabilidades del último token con el resto de tokens por los embeddings del resto de tokens

Es decir, cada fila (que correspondería a su correspondiente token, la primera fila correspondería al primer token, la segunda fila correspondería al segundo token, ...) se representa con la suma de probabilidades del token de esa fila con los anteriores por su embedding y los anteriores

Cada fila corresponde a la suma ponderada de todas las probabilidades con los embeddings consigo mismo y los anteriores tokens. **Para cada fila (lo que correspondería a cada token) se ha eliminado la información de las probabilidades y embeddings de los tokens futuros**

**Cada fila (que correspondería a cada token) solo tiene información de si mismo y los tokens anteriores**

**Se ha enmascarado el futuro**

## ¿Por qué enmascarar?

A la hora de entrenar se le da al transformer la secuencia de entrada y la secuencia de salida, no se va generando token a token. Por lo que no queremos que el transformer tenga información de los tokens futuros, porque entonces no necesitaría predecir, ya tendría la información. Por lo que no aprendería

De manera que se realiza este enmascaramiento para que el transformer se pueda entrenar con la secuencia de entrada y salida de golpe y no acceda a los tokens futuros

A la hora de hacer inferencia también se enmascara.

Supongamos que queremos traducir la frase `Me encanta el queso` al inglés. Por lo que en la secuencia de iteracciones sería esta (vamos a suponer que cada palablra equivale a un token)

01. Input to encoder: `Me encanta el queso`  
   Input to decoder: `<start>`  
   Output decoder: `I`

02. Input to encoder: `Me encanta el queso`  
   Input to decoder: `<start> I`  
   Output decoder: `love`

03. Input to encoder: `Me encanta el queso`  
   Input to decoder: `<start> I love`  
   Output decoder: `cheese`

04. Input to encoder: `Me encanta el queso`  
   Input to decoder: `<start> I love cheese`  
   Output decoder: `<end>`


Como puedes ver, en cada iteracción le entra al decoder un token más que en la iteracción anterior. Pero como hemos visto, al trabajar con matrices se suelen utilizar matrices del mismo tamaño, por lo que en realidad lo que ocurre todo el rato es esto

01. Input to encoder: `Me encanta el queso`  
   Input to decoder: `<start> <pad> <pad> <pad>`  
   Output decoder: `I`

02. Input to encoder: `Me encanta el queso`  
   Input to decoder: `<start> I <pad> <pad>`  
   Output decoder: `love`

03. Input to encoder: `Me encanta el queso`  
   Input to decoder: `<start> I love <pad>`  
   Output decoder: `cheese`

04. Input to encoder: `Me encanta el queso`  
   Input to decoder: `<start> I love cheese`  
   Output decoder: `<end>`

Todo el rato se mete una matriz del mismo tamaño con el token de `padding` en las posiciones en las que el transformer no tiene que predecir la siguiente palabra.

Así que durante la inferencia también se enmascara para que el transformer no haga cálculos de atención con los tokens de `padding` y no se desvíe de la traducción

## Implementación

En su día creamos la siguiente clase para el `Scaled Dot-Product Attention`

``` python
import torch
import torch.nn as nn

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / torch.sqrt(torch.tensor(self.dim_embedding))
        # softmax
        attention_matrix = torch.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output
```

Por lo que vamos a completarla con el enmascaramiento. Para ello vamos a utilizar la función [masked_fill](https://pytorch.org/docs/stable/generated/torch.Tensor.masked_fill.html) de Pytorch, que aplicara una máscara de $0$ en la diagonal y la parte inferior y $-\infty$ en la parte superior

$$
\begin{pmatrix}
0 & -\infty & \cdots & -\infty \\
0 & 0 & \cdots & -\infty \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 0 \\
\end{pmatrix}
$$

[masked_fill](https://pytorch.org/docs/stable/generated/torch.Tensor.masked_fill.html) lo que hace es aplicar una máscara a los elementos de una matriz de entrada.

In [14]:
import torch
import torch.nn as nn

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value, mask=None):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
            mask: mask matrix (optional)
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / torch.sqrt(torch.tensor(self.dim_embedding))
        # Mask (optional)
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        # softmax
        attention_matrix = torch.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

Viendo la línea `scale = scale.masked_fill(mask == 0, float('-inf'))` lo que hace `masked_fill` es enmascarar la matriz `scale` con un $-\infty$ en las posiciones de la máscara `mask` que sean iguales a $0$

Por lo que generaremos una máscara triangular inferior con $1$ en la diagonal y por debajo de ella y $0$ en la parte superior

$$
\begin{pmatrix}
1 & 0 & \cdots & 0 \\
1 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
1 & 1 & \cdots & 1 \\
\end{pmatrix}
$$

Y `masked_fill` lo que hará será transformar los $0$ de la máscara en $-\infty$

$$
\begin{pmatrix}
0 & -\infty & \cdots & -\infty \\
0 & 0 & \cdots & -\infty \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 0 \\
\end{pmatrix}
$$

Vamos a verlo, supongamos que tenemos la matriz resultante de `Scale`

In [15]:
import torch

scale_matrix = torch.rand(3,3)
scale_matrix

tensor([[0.5461, 0.5667, 0.8590],
        [0.6783, 0.1796, 0.8954],
        [0.5401, 0.1381, 0.5374]])

In [16]:
triangular_mask = torch.tril(torch.ones(3,3))
triangular_mask

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [17]:
masked_matrix = scale_matrix.masked_fill(triangular_mask == 0, float('-inf'))
masked_matrix

tensor([[0.5461,   -inf,   -inf],
        [0.6783, 0.1796,   -inf],
        [0.5401, 0.1381, 0.5374]])

Como se puede ver, se han colocado $-\infty$ en la parte superior de la matriz

Vamos a hacer como en el encoder, vamos a coger el resultado del embedding más el positional encoding de una sentencia con BERT y lo vamos a pasar por el `Scaled Dot-Product Attention` con y sin enmascaramiento

In [18]:
import torch
from transformers import BertModel, BertTokenizer

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

In [19]:
sentence1 = "I gave the dog a bone because it was hungry"
tokens1, input_ids1, token_embeddings1, positional_encodings1, embeddings_with_positional_encoding1 = extract_embeddings(sentence1)

In [20]:
X = embeddings_with_positional_encoding1
X.shape

torch.Size([1, 12, 768])

Creamos un objeto de la clase `Scaled Dot-Product Attention`

In [21]:
dim_embedding = X.shape[2]
scaled_dot_product_attention = ScaledDotProductAttention(dim_embedding=dim_embedding)

Obtenemos primero el resultado sin enmascaramiento

In [22]:
attention_no_mask = scaled_dot_product_attention(key=X, query=X, value=X)
attention_no_mask.shape, (attention_no_mask.detach().numpy()*100).astype(int)/100

(torch.Size([1, 12, 768]),
 array([[[ 0.07,  0.01, -0.11, ...,  0.2 ,  0.25,  0.26],
         [ 0.36, -0.13, -0.21, ...,  0.11,  0.79,  0.08],
         [ 0.2 , -0.34,  0.18, ...,  0.08,  0.33, -0.08],
         ...,
         [-0.54,  0.12,  0.05, ..., -0.33,  0.2 ,  0.5 ],
         [-0.16, -0.1 , -0.27, ...,  0.94,  0.42, -0.48],
         [ 0.73,  0.24, -0.22, ...,  0.23, -0.63, -0.48]]]))

Ahora con enmascaramiento, primero creamos la máscara `mask` que se aplicará en `masked_fill`. Creamos una matriz triangular inferior del tamaño del número de tokens

In [23]:
def create_mask(sequence_len):
    """
    Args:
        sequence_len: length of sequence
        
    Returns:
        mask matrix
    """
    mask = torch.tril(torch.ones((sequence_len, sequence_len)))
    return mask

In [24]:
batch_size = X.shape[0]
sentence1_len = X.shape[1]
mask = create_mask(X.shape[1])
mask.shape, mask.detach().numpy().astype(int)

(torch.Size([12, 12]),
 array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

Esa es la máscara que entrará en nuestro `Scaled Dot-Product Attention` de manera que `masked_fill` enmascarará la matrid `scale` con $-\infty$ en las posiciones en las que `mask` sean 0 (`scale = scale.masked_fill(mask == 0, float('-inf'))`)

In [25]:
attention_mask = scaled_dot_product_attention(key=X, query=X, value=X, mask=mask)
attention_mask.shape, (attention_mask.detach().numpy()*100).astype(int)/100

(torch.Size([1, 12, 768]),
 array([[[ 0.07,  0.01, -0.11, ...,  0.2 ,  0.24,  0.26],
         [ 0.44, -0.15, -0.26, ...,  0.12,  0.88,  0.07],
         [ 0.36, -0.45,  0.2 , ...,  0.14,  0.37, -0.18],
         ...,
         [-0.54,  0.12,  0.06, ..., -0.33,  0.2 ,  0.5 ],
         [-0.16, -0.1 , -0.27, ...,  0.95,  0.42, -0.48],
         [ 0.73,  0.24, -0.22, ...,  0.23, -0.63, -0.48]]]))

Vamos a ver las dos juntas para evr que hay diferencia

In [ ]:
print(f"Sin máscara:\n{(attention_no_mask.detach().numpy()*100).astype(int)/100}")
print(f"\nCon máscara:\n{(attention_mask.detach().numpy()*100).astype(int)/100}")

Sin máscara:  [[[ 0.07  0.01 -0.11 ...  0.2   0.25  0.26]
  [ 0.36 -0.13 -0.21 ...  0.11  0.79  0.08]
  [ 0.2  -0.34  0.18 ...  0.08  0.33 -0.08]
  ...
  [-0.54  0.12  0.05 ... -0.33  0.2   0.5 ]
  [-0.16 -0.1  -0.27 ...  0.94  0.42 -0.48]
  [ 0.73  0.24 -0.22 ...  0.23 -0.63 -0.48]]]

Con máscara:  [[[ 0.07  0.01 -0.11 ...  0.2   0.24  0.26]
  [ 0.44 -0.15 -0.26 ...  0.12  0.88  0.07]
  [ 0.36 -0.45  0.2  ...  0.14  0.37 -0.18]
  ...
  [-0.54  0.12  0.06 ... -0.33  0.2   0.5 ]
  [-0.16 -0.1  -0.27 ...  0.95  0.42 -0.48]
  [ 0.73  0.24 -0.22 ...  0.23 -0.63 -0.48]]]
